In [1]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [2]:
def calc_est(rets):
    num_succ = len(rets)
    sum_succ=0;sum_fails=0;sum_feats=0; total_wait = 0; max_succ=-1; max_fails=-1;p_all_fail = 1; markov_bound = 20; geom_all_fail=1; p_succ_sum=0; geom_bound=2;p_min=2
    max_smoothing=0.99
    for key,value in rets.items():
        trials_succ = value['trials']
        if max_succ == 0:  
            max_succ = trials_succ
        else:
            max_succ = max(max_smoothing * max_succ, trials_succ)  # exponential decay
        sum_succ += trials_succ
        sum_fails += sum(value['fails']) # sum cur fails
        max_fails = max(sum_fails,max_fails) # max of fails
        total_wait += trials_succ+sum_fails # overall since begining
        sum_feats += value['features'] # overall
    
    est_max = max_succ*num_succ # estimate looks at worst case success of a seed, multiply it by the number of seeds generated OR just successful seeds?? e
    est_avgseed = num_succ and total_wait/num_succ or 0 # sum all fails+succ trial and divide by num successes
    est_avgfeat = sum_feats and total_wait/sum_feats or 0
    est_markov = est_avgseed*markov_bound
    alpha = 1; beta = 1
    for key,value in rets.items():
        trials_succ = value['trials']
        p_this_succ = (1+alpha)/((trials_succ-1)+alpha+beta) # uninformative prior, bayesian smoothign for prob estimate
        p_this_fail = 1-p_this_succ # fail rate
        p_succ_sum += p_this_succ
        p_min = min(p_min,p_this_succ)
        p_all_fail = p_all_fail*p_this_fail # all fail chance
        geom_all_fail = geom_all_fail*(p_this_fail**geom_bound) # (1-p)^(5E(t)) , prob that all require more than 5 times expected val
        alpha+=1
        # beta+=sum(value['fails'])
        beta += (trials_succ-1)
    
    # p_geom_ident = num_succ/sum_succ # 1/bar(X)
    p_geom_ident = num_succ/total_wait # succ/total_time
    # geom_bound *= sum_succ/num_succ # bound = 20*E(X)
    geom_bound *= max_succ # bound = 5*Max trials for succ
    p_geom_ident = (1-p_geom_ident)**geom_bound # bound by 5* expected value of finding a seed?, (1-p)^bound, P(more than bound)
    p_geom_ident = 1-p_geom_ident # P(less than bound) = 1-P(more than bound)
    # p_atl = 1-(p_all_fail) # at least one succ
    p_atl = 1-((1-p_min)**num_succ) # use worst seed in at least one success?
    est_atl = num_succ/p_atl
    p_geom_nonident = 1-geom_all_fail # at least 1 require less than max time
    est_geom_nonident = p_geom_nonident and (num_succ*geom_bound)/p_geom_nonident or 0 # the time required for at least 1 seed to get succ in less than expected number of trials, can add a multiplier too
    est_geom_ident = p_geom_ident and (geom_bound)/p_geom_ident or 0
    # p_succ_cum = p_succ_cum/num_succ # each seed can be picked with equal prob, this is including overlaps, reduce later
    return est_max,est_avgseed,est_avgfeat,est_atl,est_geom_nonident,est_markov

In [12]:
def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    all_ret = {}
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        line2 = lines[i+1].strip()
        point, rest = line.split(", ", 1)  # Split only on the first ", "
        point = int(point)
        trials = int(rest)
        part2 = line2.split(", ",1)
        if len(part2)>1:
            fails = [int(x) for x in part2[1].split(",") if x.strip()]
        else:
            fails=[0]
        all_ret[int(i/8)] = {
            'point': point,
            'trials': trials,
            'fails': fails,
            'features': int(lines[i + 2].strip().split(", ")[1]),
            'llaplace': int(lines[i + 3].strip().split(", ")[1]),
            'lgt': int(lines[i + 4].strip().split(", ")[1]),
            'maxfork': int(lines[i + 5].strip().split(", ")[1]),
            'avgfork': int(lines[i + 6].strip().split(", ")[1]),
            'rate': int(lines[i + 7].strip().split(", ")[1])
        }
        i += 8
    return all_ret

def generate_csv(file_name):
    file_path = f'data/{file_name}.txt'
    all_ret = parse_file(file_path)
    trials_to_succ = 0;inputs = [];overall_trials = 0;overall_features = 0;all_ret_so_far={}
    # history_range = range(20,161,20) ## look at past discoveries?
    # history_range = [] ## look at past discoveries?

    for i in range(len(all_ret)):
        succ_trials = all_ret[i]['trials'] # in OG run what is the number trials it took from all seeds which are being reset between every 2 discoveries, same as point[i]-point[i-1]
        fails = sum(all_ret[i]['fails']) # fails vect
        trials_to_succ = 0
        trials_to_succ = fails+succ_trials # sum both gives total btw 2 disc
        features = all_ret[i]['features']
        overall_trials = overall_trials+trials_to_succ
        overall_features = overall_trials+features
        rate = all_ret[i]['rate']
        all_ret_so_far[i] = all_ret[i]
        estimates=[]
        if(i!=0):
            maxfork = all_ret[i-1]['maxfork']
            avgfork = all_ret[i-1]['avgfork']
            # for h in history_range: ## when im at point i=10, need to use values 7,8,9 if h=3, when at i=1, need to use values 0 for h=1, and none from h=2
            #     if i < h: # 1<2
            #         ests = [0,0,0,0,0,0]
            #     else:
            #         ## h=2, i=10 : all_ret[(i-h:(i-1)] [8,9], h=1, i=1 : all_ret[0:i-1]
            #         hist_ret = {k: all_ret[k] for k in range(i-h, i) if k in all_ret}
            #         ests = calc_est(hist_ret)
            #     ests = [round(est) for est in ests]
            #     estimates.extend(ests)
            ests = calc_est(all_ret_so_far)
            ests = [round(est) for est in ests]
            estimates.extend(ests)
        else:
            maxfork = 0
            avgfork = 0
            estimates = [0,0,0,0,0,0]

        ######### Shift the fork max/avg values down by one because we fork after finding the seed in GB############
        # seed_limit = 1; fork_limit = 10; # _ ---> doing this
        est_ll = all_ret[i]['llaplace']
        est_lgt = all_ret[i]['lgt']
        defaults = [features,overall_trials,overall_features,succ_trials,trials_to_succ,maxfork,avgfork,rate,est_ll,est_lgt]
        inputs.append([i] + defaults + estimates) # appending the estimate for point i made from i-1 and before, comparing against trials at point i, and 
        
    headers = ["Index","Features","OverallTrials","OverallFeatures","SuccTrials","TrialsToDisc","MaxFork","AvgFork", "Rate", "LLaplace","LGT"]
    # for h in history_range:
        # headers.extend([f"MaxTrials_{h}",f"AvPath_{h}",f"AvFeat_{h}",f"ATL_{h}",f"Extr_{h}",f"Markov_{h}"])
    headers.extend([f"MaxTrials_all",f"AvPath_all",f"AvFeat_all",f"ATL_all",f"Extr_all",f"Markov_all"])
    with open(f"./data/csv/{file_name}.csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers)
            writer.writerows(inputs)

In [4]:
def get_uests(true_col, est_col):
    true_col = pd.Series(true_col)
    est_col = pd.Series(est_col)
    underestimates = (true_col>est_col)  # Boolean mask for underestimations
    cnt_uest = ((np.nansum(underestimates.astype(int))) / len(underestimates)) * 100
    diff = true_col - est_col  # Raw difference
    diff = np.where(np.isfinite(diff), diff, 0)
    if diff[underestimates].size > 0:
        max_uest = diff[underestimates].max()
    else:
        max_uest = 1
    rel_diff = (diff / true_col) * 100  # Percentage error
    if rel_diff[underestimates].size > 0:
        rel = rel_diff[underestimates].mean()
    else:
        rel = 0
    non_zero_diff = diff[underestimates][diff[underestimates] != 0]
    if len(non_zero_diff) > 0:
        min_uest = non_zero_diff.min()
    else:
        min_uest = 1

    return cnt_uest,max_uest,min_uest,rel

def get_oests(true_col, est_col):
    true_col = pd.Series(true_col)
    est_col = pd.Series(est_col)
    overestimates = ((true_col<est_col) & (true_col != 0))
    diff = est_col-true_col
    rel_diff = (diff / true_col) * 100 
    # print("Diff:",diff,"True:",true_col,"rel diff:",rel_diff)
    if diff[overestimates].size > 0:
        max_oest = diff[overestimates].max()
    else:
        max_oest = 1
    severe = diff>4000 # worse than 4000 s of overprediction?
    if diff[severe].size > 0:
        severe_overestimation = diff[severe].mean()
    else:
        severe_overestimation = 1

    return max_oest,severe_overestimation,rel_diff[overestimates].mean()

# Concordance correlation coefficient
def get_ccc(true_col, est_col):
    true_col = pd.Series(true_col)
    est_col = pd.Series(est_col)
    data = pd.concat([true_col, est_col], axis=1)
    data = data.replace([np.inf, -np.inf], np.nan).dropna()
    true_col = data.iloc[:, 0]
    est_col = data.iloc[:, 1]
    if len(true_col) == 0:
        return np.nan
    mean_true = true_col.mean()
    mean_est = est_col.mean()
    var_true = true_col.var()
    var_est = est_col.var()
    std_true = np.sqrt(var_true)
    std_est = np.sqrt(var_est)
    correlation = true_col.corr(est_col)
    ccc = (2 * correlation * std_true * std_est) / (
        var_true + var_est + (mean_true - mean_est) ** 2
    )
    return ccc

def get_smape(y_true, y_pred, under_weight=2, epsilon=1e-8):
    """
    Compute Symmetric Mean Absolute Percentage Error (SMAPE).
    Parameters:
    - y_true: array-like of true values
    - y_pred: array-like of predicted values
    - epsilon: small value to avoid division by zero
    Returns:
    - smape_value: float, SMAPE in percentage (0 to 200%)
    """
    y_true = pd.Series(y_true)
    y_pred = pd.Series(y_pred)
    data = pd.concat([y_true, y_pred], axis=1)
    data = data.replace([np.inf, -np.inf], np.nan).dropna()
    y_true = data.iloc[:, 0]
    y_pred = data.iloc[:, 1]
    if len(y_true) == 0:
        return np.nan

    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    denominator = np.where(denominator == 0, epsilon, denominator)
    errors = y_pred - y_true
    weights = np.where(errors < 0, under_weight, 1.0)
    smape_values = weights*np.abs(errors) / denominator
    return 100 * np.mean(smape_values)

In [21]:
def get_col(method, n, estimates):
        if method == 'ATL':
            return estimates.filter(regex=f'ATL_{n}$').values.flatten()
        elif method == 'AvPath':
            return estimates.filter(regex=f'AvPath_{n}$').values.flatten()
        elif method == 'AvFeat':
            return estimates.filter(regex=f'AvFeat_{n}$').values.flatten()
        elif method == 'MaxTrials':
            return estimates.filter(regex=f'MaxTrials_{n}$').values.flatten()
        elif method == 'Markov':
            return estimates.filter(regex=f'Markov_{n}$').values.flatten()
        elif method == 'Extr':
            return estimates.filter(regex=f'Extr_{n}$').values.flatten()
        elif method == 'LLaplace':
            return estimates.filter(regex=f'LLaplace$').values.flatten()
        elif method == 'LGT':
            return estimates.filter(regex=f'LGT$').values.flatten()    
        elif method == 'Rate':
            return estimates.filter(regex=f'Rate$').values.flatten()
        
def plot_stats(file_name, plot_type):
    df = pd.read_csv(f'./data/csv/{file_name}.csv')
    columns = df.columns.tolist()
    orig_values = list(df['TrialsToDisc'])
    max_fork_values = list(df['MaxFork'])
    avg_fork_values = list(df['AvgFork'])
    estimates = df[columns[8:]] ## dont hardcode
    # remove = ["ExWait"]
    # pattern = '|'.join(remove)
    # cols_remove = estimates.filter(regex=pattern).columns
    # estimates = estimates.drop(columns=cols_remove)
    cols_dict = {
        'LLaplace': get_col('LLaplace', 0,estimates),
        'LGT': get_col('LGT', 0,estimates),
        'Rate': get_col('Rate',0,estimates),
    }
    rate = np.array(cols_dict['Rate'], dtype=object)
    rate = pd.to_numeric(rate, errors='coerce')
    rate = np.nan_to_num(rate, nan=1.0, posinf=1.0, neginf=1.0)
    
    if plot_type == "errors":
        #### Risk averse Errors plots ########
        # single metric #
        smape_dict = {}
        for col in estimates.columns:
            if col!= 'Rate' and col!="":
                # ccc = get_ccc(max_fork_values/rate,estimates[col]/rate)
                smape = get_smape(max_fork_values/rate,estimates[col]/rate)
                smape_dict[col] = smape
                # print(col,ccc)
        # fig, (ax1,ax2,ax3) = plt.subplots(3, 1, figsize=(16, 16))
        # vals = list(cnt_uest_dict.values())
        # ax1.bar(cnt_uest_dict.keys(),vals)
        # ax1.set_title('Uest Counts', fontsize=16)
        # ax1.set_xlabel('Estimate', fontsize=16)
        # ax1.set_ylabel('Counts %', fontsize=16)
        # ax1.tick_params(axis='x', rotation=45,labelsize=16)
        
        # Uest #
        cnt_uest_dict = {}
        max_dict = {}
        min_dict = {}
        rel_uest_avg_dict = {}
        for col in estimates.columns:
            if col!= 'Rate' and col!="":
                cnt,max_uest,min_uest,rel_avg = get_uests(max_fork_values/rate,estimates[col]/rate)
                cnt_uest_dict[col] = cnt
                max_dict[col] = np.log10(max_uest)
                min_dict[col] = np.log10(min_uest)
                rel_uest_avg_dict[col] = rel_avg
        # fig, (ax1,ax2,ax3) = plt.subplots(3, 1, figsize=(16, 16))
        # vals = list(cnt_uest_dict.values())
        # ax1.bar(cnt_uest_dict.keys(),vals)
        # ax1.set_title('Uest Counts', fontsize=16)
        # ax1.set_xlabel('Estimate', fontsize=16)
        # ax1.set_ylabel('Counts %', fontsize=16)
        # ax1.tick_params(axis='x', rotation=45,labelsize=16)
        
        # max_vals = list(max_dict.values())
        # max_vals = [0 if np.isnan(i) else i for i in max_vals]
        # # max_vals = np.log10(max_vals)
        # min_vals = list(min_dict.values())
        # min_vals = [0 if np.isnan(i) else i for i in min_vals]
        # # min_vals = np.log10(min_vals)
        # bar_width = 0.4
        # x = np.arange(len(max_dict.keys()))
        # ax2.bar(x - bar_width/2, max_vals, width=bar_width, label='Max Uest')
        # ax2.bar(x + bar_width/2, min_vals, width=bar_width, label='Min Uest')
        # ax2.set_title("Max/Min Uest", fontsize=16)
        # ax2.set_xlabel("Estimate", fontsize=16)
        # ax2.set_ylabel("log(sec)", fontsize=16)
        # ax2.set_xticks(x)
        # ax2.legend()
        # ax2.set_xticklabels(max_dict.keys(), rotation=45)
        # ax2.tick_params(axis='x', rotation=45,labelsize=16)
        
        # vals = list(rel_uest_avg_dict.values())
        # vals = [0 if np.isnan(i) else i for i in vals]
        # ax3.bar(rel_uest_avg_dict.keys(),vals)
        # ax3.set_title("Relative Avg Uest", fontsize=16)
        # ax3.set_ylabel("%", fontsize=16)
        # ax3.set_xlabel("Estimate", fontsize=16)
        # ax3.tick_params(axis='x', rotation=45,labelsize=16)
        # plt.tight_layout()
        # os.makedirs(f'./plots/errors/risk_aver/uest/', exist_ok=True)
        # plt.savefig(f'./plots/errors/risk_aver/uest/{file_name}_err_risk_aver.png', dpi=300, bbox_inches='tight')
        # plt.clf()
        
        # Oest #
        max_oest_dict = {}
        severe_avg_dict = {}
        rel_avg_dict = {}
        for col in estimates.columns:
            if col!= 'Rate' and col != "":  
                max, severe,rel = get_oests(max_fork_values/rate,estimates[col]/rate)
                max_oest_dict[col] = np.log10(max) if max else 0
                severe_avg_dict[col] = np.log10(severe) if severe else 0
                rel_avg_dict[col] = np.log10(rel) if rel else 0
        # fig, (ax1,ax2,ax3) = plt.subplots(3, 1, figsize=(16, 16))
        # vals = list(max_oest_dict.values())
        # # vals = np.log10(vals)
        # vals = [0 if np.isnan(i) else i for i in vals]
        # ax1.bar(max_oest_dict.keys(),vals)
        # ax1.set_title("Max Oest", fontsize=16)
        # ax1.set_ylabel("log(sec)", fontsize=16)
        # ax1.set_xlabel("Estimate", fontsize=16)
        # ax1.tick_params(axis='x', rotation=45,labelsize=16)
        
        # vals = list(severe_avg_dict.values())
        # vals = [0 if np.isnan(i) else i for i in vals]
        # # vals = np.log10(vals)
        # ax2.bar(severe_avg_dict.keys(),vals)
        # ax2.set_title("Oest (>1000s) Sum", fontsize=16)
        # ax2.set_ylabel("log(sec)", fontsize=16)
        # ax2.set_xlabel("Estimate", fontsize=16)
        # ax2.tick_params(axis='x', rotation=45,labelsize=16)
        
        # vals = list(rel_avg_dict.values())
        # vals = [0 if np.isnan(i) else i for i in vals]
        # # vals = np.log10(vals)
        # ax3.bar(rel_avg_dict.keys(),vals)
        # ax3.set_title("Relative Avg Oest", fontsize=16)
        # ax3.set_ylabel("log(%)", fontsize=16)
        # ax3.set_xlabel("Estimate", fontsize=16)
        # ax3.tick_params(axis='x', rotation=45,labelsize=16)
        # plt.tight_layout()
        # os.makedirs(f'./plots/errors/risk_aver/oest/', exist_ok=True)
        # plt.savefig(f'./plots/errors/risk_aver/oest/{file_name}_err_risk_aver.png', dpi=300, bbox_inches='tight')
        # plt.clf()
        
        return cnt_uest_dict,max_dict,min_dict,rel_uest_avg_dict,max_oest_dict,severe_avg_dict,rel_avg_dict, smape_dict

        ##### Risk netural Errors plots ########
        # max_uest_percent_dict = {}
        # max_overest_percent_dict = {}
        # avg_overest_percent_dict = {}
        # for col in estimates.columns:
        #     if col=="Rate":
        #         continue
        #     uest_percent,max_overest_percent, avg_overest_percent = get_errors(avg_fork_values,estimates[col])
        #     max_uest_percent_dict[col] = uest_percent
        #     max_overest_percent_dict[col] = max_overest_percent
        #     avg_overest_percent_dict[col] = avg_overest_percent
        # fig, (ax1,ax2,ax3) = plt.subplots(3, 1, figsize=(16, 16))
        # vals = list(max_uest_percent_dict.values())
        # # vals = np.log10(vals)
        # ax1.bar(max_uest_percent_dict.keys(),vals)
        # ax1.set_title('Avg Underest %', fontsize=10)
        # ax1.set_xlabel('Estimate', fontsize=10)
        # ax1.set_ylabel('Percent', fontsize=10)
        # ax1.tick_params(axis='x', rotation=90)
        # vals = list(max_overest_percent_dict.values())
        # vals = np.log10(vals)
        # ax2.bar(max_overest_percent_dict.keys(),vals)
        # ax2.set_title("Max Overest %", fontsize=10)
        # ax2.set_ylabel("log(Percent)", fontsize=10)
        # ax2.set_xlabel("Estimate", fontsize=10)
        # ax2.tick_params(axis='x', rotation=90)
        # vals = list(avg_overest_percent_dict.values())
        # vals = np.log10(vals)
        # ax3.bar(avg_overest_percent_dict.keys(),vals)
        # ax3.set_title("Avg Overest %", fontsize=10)
        # ax3.set_ylabel("log(Percent)", fontsize=10)
        # ax3.set_xlabel("Estimate", fontsize=10)
        # ax3.tick_params(axis='x', rotation=90)
        # plt.tight_layout()
        # plt.savefig(f'./plots/errors/risk_neut/{file_name}_err_risk_neut.png', dpi=300, bbox_inches='tight')
        # plt.clf()

    elif plot_type == "true":
        ### True value plots ###
        # x=np.arange(len(orig_values))
        # ax1 = plt.gca()
        # ax1.set_ylabel("log(Trials)", fontsize=12)
        # ax1.scatter(x,np.log10(orig_values), color='black', alpha=0.5,label='Orignal Vals', marker='x')
        # ax1.scatter(x,np.log10(avg_fork_values), color='blue', alpha=0.5,label='Avg Fork Vals', marker='x')
        # ax1.scatter(x,np.log10(max_fork_values), color='red', alpha=0.5,label='Max Fork Vals', marker='x')
        # plt.xlabel("Index", fontsize=12)
        # plt.grid(True,linestyle='--',alpha=0.7)
        # plt.legend(loc='upper left')
        # plt.savefig(f'{file_name}_trials.png', dpi=300, bbox_inches='tight')
        # plt.clf()

        ### True value Time plots ###
        # x=np.arange(len(orig_values))
        # ax1 = plt.gca()
        # ax1.set_ylabel("log(Time)", fontsize=12)
        # time_true_values = np.where(cols_dict['Rate'] != 0, orig_values / cols_dict['Rate'], 0)
        # time_max_true_values = np.where(cols_dict['Rate'] != 0, max_fork_values / cols_dict['Rate'], 0)
        # time_avg_true_values = np.where(cols_dict['Rate'] != 0, avg_fork_values / cols_dict['Rate'], 0)
        # ax1.scatter(x,np.log10(time_true_values), color='black', alpha=0.5,label='Original Vals', marker='x')
        # ax1.scatter(x,np.log10(time_avg_true_values), color='blue', alpha=0.5,label='AvgFork Vals', marker='x')
        # ax1.scatter(x,np.log10(time_max_true_values), color='red', alpha=0.5,label='MaxFork Vals', marker='x')
        # plt.xlabel("Index", fontsize=12)
        # plt.grid(True,linestyle='--',alpha=0.7)
        # plt.legend(loc='upper left')
        # plt.savefig(f'{file_name}_time.png', dpi=300, bbox_inches='tight')
        # plt.clf()
        pass

    elif plot_type == "ests":    
        #### Estimators ######
        for est_type in ["baseline","novel1","novel2"]:
            hist = "all"
            rate = np.array(cols_dict['Rate'], dtype=object)
            rate = pd.to_numeric(rate, errors='coerce')
            rate = np.nan_to_num(rate, nan=1.0, posinf=1.0, neginf=1.0)

            if est_type == "baseline":
                time_AvSeed = np.where(rate != 0, get_col('AvPath',hist,estimates) / rate, 0)
                time_AvFeat = np.where(rate != 0, get_col('AvFeat',hist,estimates) / rate, 0)
                time_Markov = np.where(rate != 0, get_col('Markov',hist,estimates) / rate, 0)
            elif est_type == "novel1":
                time_max = np.where(rate != 0, get_col('MaxTrials',hist,estimates) / rate, 0)
            elif est_type == "novel2":
                time_atl = np.where(rate != 0, get_col('ATL',hist,estimates) / rate, 0)
                time_geom_nonident = np.where(rate != 0, get_col('Extr',hist,estimates) / rate, 0)
            time_LLaplace = np.where(rate != 0, cols_dict['LLaplace'] / rate, 0)
            time_LGT = np.where(rate != 0, cols_dict['LGT'] / rate, 0)
            time_max_true_values = np.where(rate != 0, max_fork_values / rate, 0)
            # time_avg_true_values = np.where(rate != 0, avg_fork_values / rate, 0)
            # time_orig_values = np.where(rate != 0, orig_values / rate, 0)
            
            ## Risk averse Estimators Time plots ###
            x=np.arange(len(orig_values))
            ax1 = plt.gca()
            if est_type == "novel1":
                ax1.scatter(x,np.log10(time_max),color='green',alpha=0.2,label=f'Max Trials {hist}')
                ax1.scatter(x,np.log10(time_Markov),color='red',alpha=0.2,label=f'Markov 95% {hist}')
            if est_type == "novel2":
                ax1.scatter(x,np.log10(time_geom_nonident),color='deeppink',alpha=0.2,label=f'Extr. Val Set of Seeds {hist}')
                ax1.scatter(x,np.log10(time_atl),color='blue',alpha=0.2,label=f'ATL {hist}')
            if est_type == "baseline":                 
                ax1.scatter(x,np.log10(time_AvSeed),color='lightblue',alpha=0.2,label=f'AvSeed {hist}')
                ax1.scatter(x,np.log10(time_AvFeat),color='lightgreen',alpha=0.2,label=f'AvFeat {hist}')
            ax1.scatter(x,np.log10(time_LLaplace),color='yellow',alpha=0.2,label='LLaplace')
            ax1.scatter(x,np.log10(time_LGT),color='cyan',alpha=0.2,label='LGT')
            ax1.scatter(x,np.log10(time_max_true_values), color='black', alpha=0.2,label='MaxFork Vals', marker='x')
            ax1.legend(loc='upper left',fontsize=5)
            ax1.set_ylabel("log(sec)", fontsize=10)
            plt.xlabel("Discoveries", fontsize=10)
            plt.grid(True,linestyle='--',alpha=0.7)
            os.makedirs(f'./plots/est_risk_aver/{est_type}/{hist}', exist_ok=True)
            plt.savefig(f'./plots/est_risk_aver/{est_type}/{hist}/{file_name}_est_risk_aver.png', dpi=300, bbox_inches='tight')
            plt.clf()

            ### Risk neutral Estimators Time plots ###
            # x=np.arange(len(orig_values))
            # ax1 = plt.gca()
            # if est_type == "novel1":
            #     ax1.scatter(x,np.log10(time_max),color='green',alpha=0.2,label=f'Max {hist}')
            #     ax1.scatter(x,np.log10(time_atl),color='blue',alpha=0.2,label=f'ATL {hist}')
            # if est_type == "novel2":
            #     ax1.scatter(x,np.log10(time_geom_nonident),color='deeppink',alpha=0.2,label=f'Geom Non-Ident {hist}')
            #     ax1.scatter(x,np.log10(time_geom_ident),color='coral',alpha=0.2,label=f'Geom Ident {hist}')
            # if est_type == "baseline":                 
            #     ax1.scatter(x,np.log10(time_AvSeed),color='lightblue',alpha=0.2,label=f'AvSeed {hist}')
            #     ax1.scatter(x,np.log10(time_AvFeat),color='lightgreen',alpha=0.2,label=f'AvFeat {hist}')
            #     ax1.scatter(x,np.log10(time_Markov),color='red',alpha=0.2,label=f'Markov 95% {hist}')
            # ax1.scatter(x,np.log10(time_LLaplace),color='yellow',alpha=0.2,label='LLaplace')
            # ax1.scatter(x,np.log10(time_LGT),color='cyan',alpha=0.2,label='LGT')
            # ax1.scatter(x,np.log10(time_avg_true_values), color='black', alpha=0.2,label='AvgFork Vals', marker='x')
            # ax1.legend(loc='upper left',fontsize=5)
            # ax1.set_ylabel("log(sec)", fontsize=10)
            # plt.xlabel("Discoveries", fontsize=10)
            # plt.grid(True,linestyle='--',alpha=0.7)
            # os.makedirs(f'./plots/est_risk_neut/{est_type}/{hist}', exist_ok=True)
            # plt.savefig(f'./plots/est_risk_neut/{est_type}/{hist}/{file_name}_est_risk_neut.png', dpi=300, bbox_inches='tight')
            # plt.clf()

            ### Original Estimators Time plots ###
            # x=np.arange(len(orig_values))
            # ax1 = plt.gca()
            # if est_type == "novel1":
            #     ax1.scatter(x,np.log10(time_max),color='green',alpha=0.2,label=f'Max {hist}')
            #     ax1.scatter(x,np.log10(time_atl),color='blue',alpha=0.2,label=f'ATL {hist}')
            # if est_type == "novel2":
            #     ax1.scatter(x,np.log10(time_geom_nonident),color='deeppink',alpha=0.2,label=f'Geom Non-Ident {hist}')
            #     ax1.scatter(x,np.log10(time_geom_ident),color='coral',alpha=0.2,label=f'Geom Ident {hist}')
            # if est_type == "baseline":                 
            #     ax1.scatter(x,np.log10(time_AvSeed),color='lightblue',alpha=0.2,label=f'AvSeed {hist}')
            #     ax1.scatter(x,np.log10(time_AvFeat),color='lightgreen',alpha=0.2,label=f'AvFeat {hist}')
            #     ax1.scatter(x,np.log10(time_Markov),color='red',alpha=0.2,label=f'Markov 95% {hist}')
            # ax1.scatter(x,np.log10(time_LLaplace),color='yellow',alpha=0.2,label='LLaplace')
            # ax1.scatter(x,np.log10(time_LGT),color='cyan',alpha=0.2,label='LGT')
            # ax1.scatter(x,np.log10(time_orig_values), color='black', alpha=0.2,label='Orig Vals', marker='x')
            # ax1.legend(loc='upper left',fontsize=5)
            # ax1.set_ylabel("log(sec)", fontsize=10)
            # plt.xlabel("Discoveries", fontsize=10)
            # plt.grid(True,linestyle='--',alpha=0.7)
            # os.makedirs(f'./plots/est_orig/{est_type}/{hist}', exist_ok=True)
            # plt.savefig(f'./plots/est_orig/{est_type}/{hist}/{file_name}_est_orig.png', dpi=300, bbox_inches='tight')
            # plt.clf()

    #     ### Succ Val Freqs ###
    # ax1 = plt.gca()
    # ax1.hist(succ_values,bins=50,label=f'Succ Trials')
    # ax1.legend(loc='upper left',fontsize=5)
    # ax1.set_ylabel("Frequency", fontsize=10)
    # plt.xlabel("Succ Trials", fontsize=10)
    # plt.grid(True,linestyle='--',alpha=0.7)
    # plt.savefig(f'{file_name}_succ_freq.png', dpi=300, bbox_inches='tight')
    # plt.clf()

def combine_errors(aggregated_errors):
    # pattern = re.compile(r'AvPath')
    for error in aggregated_errors.items():
        # filtered_keys = [key for key in error[1].keys() if re.search(pattern, key)]
        # if filtered_keys:
        if True:
            fig,ax = plt.subplots(1,1,figsize=(12, 6))
            ax.set_ylabel(error[0],fontsize=14)
            ax.set_xlabel("Estimator",fontsize=14)
            ax.tick_params(axis='x', rotation=90,labelsize=14)
            ax.tick_params(axis='y', labelsize=14)
            ax.set_title(error[0], fontsize=14)
            bars = ax.bar(error[1].keys(), error[1].values())
            # bars =ax.bar(filtered_keys, [error[1][key] for key in filtered_keys])
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width() / 2, height, f"{height:.2f}", ha='center', va='bottom')
            plt.tight_layout()
            plt.savefig(f'./plots/errors/risk_aver/{error[0]}.jpg', dpi=300, bbox_inches='tight')
            plt.clf()

def combine_ests(file_names):
    n_files = len(file_names)
    n_cols = 3  
    n_rows = (n_files + n_cols - 1) // n_cols 

    for est_type in ["baseline","novel1","novel2"]:
        fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*3.3, n_rows*3.5))
        for i, file_name in enumerate(file_names):
            df = pd.read_csv(f'./data/csv/{file_name}.csv') # cache this
            columns = df.columns.tolist()
            max_fork_values = list(df['MaxFork'])
            estimates = df[columns[8:]] ## dont hardcode
            cols_dict = {
                'LLaplace': get_col('LLaplace', 0,estimates),
                'LGT': get_col('LGT', 0,estimates),
                'Rate': get_col('Rate',0,estimates),
            }
            hist = "all"
            rate = np.array(cols_dict['Rate'], dtype=object)
            rate = pd.to_numeric(rate, errors='coerce')
            rate = np.nan_to_num(rate, nan=1.0, posinf=1.0, neginf=1.0)

            if est_type == "baseline":
                time_AvSeed = np.where(rate != 0, get_col('AvPath',hist,estimates) / rate, 0)
                time_AvFeat = np.where(rate != 0, get_col('AvFeat',hist,estimates) / rate, 0)
            elif est_type == "novel1":
                time_Markov = np.where(rate != 0, get_col('Markov',hist,estimates) / rate, 0)
                time_max = np.where(rate != 0, get_col('MaxTrials',hist,estimates) / rate, 0)
            elif est_type == "novel2":
                time_atl = np.where(rate != 0, get_col('ATL',hist,estimates) / rate, 0)
                time_geom_nonident = np.where(rate != 0, get_col('Extr',hist,estimates) / rate, 0)
                # geomexp = np.array(get_col('GeomIdent', hist,estimates), dtype=object)
                # geomexp = pd.to_numeric(geomexp, errors='coerce')
                # geomexp = np.nan_to_num(geomexp, nan=0.0)
                # time_geom_ident = np.divide(geomexp, rate, where=rate != 0, out=np.zeros_like(rate, dtype=float))
            time_LLaplace = np.where(rate != 0, cols_dict['LLaplace'] / rate, 0)
            time_LGT = np.where(rate != 0, cols_dict['LGT'] / rate, 0)
            time_max_true_values = np.where(rate != 0, max_fork_values / rate, 0)
            row = i // n_cols
            col = i % n_cols
            ax = axs[row, col]
            
            x = np.arange(len(max_fork_values))  
            if est_type == "novel1":
                l1 = ax.scatter(x, np.log10(time_max), color='green', alpha=0.5, label=f'Max Trial')
                l2 = ax.scatter(x, np.log10(time_Markov), color='red', alpha=0.5, label=f'Markov 95%')
            if est_type == "novel2":
                l1 = ax.scatter(x, np.log10(time_geom_nonident), color='deeppink', alpha=0.5, label=f'Extr. Val Set of Seeds')
                # l2 = ax.scatter(x, np.log10(time_geom_ident), color='coral', alpha=0.5, label=f'Extr. Val Single Seed')
                l2 = ax.scatter(x, np.log10(time_atl), color='blue', alpha=0.5, label=f'At Least One Succ')
            if est_type == "baseline":                 
                l1 = ax.scatter(x, np.log10(time_AvSeed), color='lightblue', alpha=0.5, label=f'Avg Path Disc')
                l2 = ax.scatter(x, np.log10(time_AvFeat), color='lightgreen', alpha=0.5, label=f'Avg Feat Disc')
            l3 = ax.scatter(x, np.log10(time_LLaplace), color='yellow', alpha=0.5, label='LLaplace')
            l4 = ax.scatter(x, np.log10(time_LGT), color='cyan', alpha=0.5, label='LGT')
            l5 = ax.scatter(x, np.log10(time_max_true_values), color='black', alpha=0.2, label='Max Fork Vals', marker='x')
            if i == 0:  # Only add once
                handles = [l1, l2, l3 , l4, l5]
                labels = [l.get_label() for l in handles if l is not None]
            ax.grid(True, linestyle='--', alpha=0.7)
            ax.set_title(file_name)

        if n_files < n_rows * n_cols:
            for i in range(n_files, n_rows * n_cols):
                row = i // n_cols
                col = i % n_cols
                axs[row, col].axis('off')
        fig.supxlabel('Discoveries', fontsize=14)
        fig.supylabel('log(sec)', fontsize=14)
        fig.legend(handles=[h for h in handles if h is not None], 
           labels=[l for l in labels if l], 
           loc='lower center', 
           bbox_to_anchor=(0.5, -0.07), 
           ncol=len(labels))
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        os.makedirs(f'./plots/est_risk_aver/grid/', exist_ok=True)
        plt.savefig(f'./plots/est_risk_aver/grid/{est_type}.jpg', dpi=300, bbox_inches='tight')
        plt.clf()

In [22]:
# file_names = ['json','zlib','curl','libxml','openh','freetype2']
file_names = ['json.1','zlib.1','libpcap.1','libxml.1','libpng.1','freetype2.1']
# file_names = ['libpcap.1']
aggregated_errors  = {}
for file_name in file_names:
    # generate_csv(file_name)
    # plot_stats(file_name,"ests")
    cnt_uest_dict,max_dict,min_dict,rel_uest_avg_dict,max_oest_dict,severe_avg_dict,rel_avg_dict,smape_dict = plot_stats(file_name,"errors")
    dicts_to_aggregate = {
    'Count Uest': ('Uest Count %', cnt_uest_dict),
    'Max Uest': ('Max Uest log(s)', max_dict),
    'Min Uest': ('Min Uest log(s)', min_dict),
    'Relative Uest': ('Relative Uest %', rel_uest_avg_dict),
    'Max Oest': ('Max Oest log(s)', max_oest_dict),
    'Severe Oest': ('Severe Oest Sum log(s)', severe_avg_dict),
    'Relative Oest': ('Relative Oest log(%)', rel_avg_dict),
    'SMAPE': ('SMAPE',smape_dict)
    }

    for aggregate_key, dict_to_process in dicts_to_aggregate.values():
        for estimator, value in dict_to_process.items():
            aggregated_errors.setdefault(aggregate_key, {}).setdefault(estimator, 0)
            aggregated_errors[aggregate_key][estimator] += (value)/len(file_names)
# combine_ests(file_names)
combine_errors(aggregated_errors)

C:\Users\rahul\AppData\Local\Temp\ipykernel_17924\1720083501.py:48: RuntimeWarning: divide by zero encountered in divide
  smape = get_smape(max_fork_values/rate,estimates[col]/rate)
C:\Users\rahul\AppData\Local\Temp\ipykernel_17924\1720083501.py:48: RuntimeWarning: invalid value encountered in divide
  smape = get_smape(max_fork_values/rate,estimates[col]/rate)
C:\Users\rahul\AppData\Local\Temp\ipykernel_17924\1720083501.py:48: RuntimeWarning: divide by zero encountered in divide
  smape = get_smape(max_fork_values/rate,estimates[col]/rate)
C:\Users\rahul\AppData\Local\Temp\ipykernel_17924\1720083501.py:48: RuntimeWarning: invalid value encountered in divide
  smape = get_smape(max_fork_values/rate,estimates[col]/rate)
C:\Users\rahul\AppData\Local\Temp\ipykernel_17924\1720083501.py:48: RuntimeWarning: divide by zero encountered in divide
  smape = get_smape(max_fork_values/rate,estimates[col]/rate)
C:\Users\rahul\AppData\Local\Temp\ipykernel_17924\1720083501.py:48: RuntimeWarning: inv

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>